In [2]:
import pandas as pd
import json 
import os 
import hana_ml
from hana_ml import ConnectionContext
import hana_ml.dataframe as dataframe

In [16]:
data = pd.read_csv('/Users/sierra/Desktop/240905_enablement_session_prep/Assignment/Enablement-hands-on/data/data.csv', on_bad_lines='skip')
print(data.columns)

Index(['FILENAME', 'TEXT'], dtype='object')


### Create Table

In [4]:
with open(os.path.join(os.getcwd(), '/Users/sierra/Desktop/240905_enablement_session_prep/Assignment/Enablement-hands-on/secrets/HANACloudServiceKey.json')) as f:
    hana_env_c = json.load(f)
    port_c  = hana_env_c['port']
    user_c  = hana_env_c['user']
    url_c  = hana_env_c['url']
    pwd_c  = hana_env_c['pwd']

conn = dataframe.ConnectionContext(
    address=url_c,  
    port=443,
    user=user_c,
    password=pwd_c,
    encrypt='true'
)

cursor = conn.connection.cursor()
cursor.execute("""SET SCHEMA GEN_AI""")

True

In [11]:
conn.get_current_schema()

'GEN_AI'

In [12]:
SCHEMA_NAME = "GEN_AI"  
TABLE_NAME  = "CUSTOMER_REVIEWS_1K_SIERRA" 

if not conn.has_table(table=TABLE_NAME, schema=SCHEMA_NAME):
    conn.create_table(table=TABLE_NAME, schema=SCHEMA_NAME, table_structure={'FILENAME':'NVARCHAR(100)','TEXT':'NCLOB','VECTOR':'REAL_VECTOR(1536)'})

In [17]:
data2=data.copy()

# import dataframe into hana table
from hana_ml.dataframe import create_dataframe_from_pandas
v_hdf = create_dataframe_from_pandas(
    connection_context=conn,
    pandas_df=data2, 
    table_name="CUSTOMER_REVIEWS_1K_SIERRA", 
    allow_bigint=True, 
    append=True,
    force=False)

100%|██████████| 1/1 [00:00<00:00, 15.62it/s]


In [18]:
hdf = conn.sql('''SELECT TOP 10
             "FILENAME",
             "TEXT"
             FROM CUSTOMER_REVIEWS_1K_SIERRA''')
df_abstract = hdf.collect()
df_abstract.head()

,FILENAME,TEXT
0,customer251.json,Absolutely wonderful food & service! Go there ...
1,customer252.json,Pretty slow service and the waitresses aren't ...
2,customer253.json,I recently had the pre fixe meal for $22 and I...
3,customer254.json,It's no different than any other Chinese place...
4,customer255.json,Finally got to eat here last night. Eclectic f...
